## Trending Topics - A Drill

In this drill, we are going to continue our look at "trends." What do we think it means for something to trend? For a trend to sweep across the nation or around the globe? Can we say something is trendy or in fashion? Or are we talking about trendsetting?  

In social media platforms, trending is often reduced to popularity - a topic is being discussed at some time, in some region, perhaps among a certain community of people. So there is an implicit assumption that a trend is bigger than you and your posts, perhaps bigger than your city, but is something that is characteristic of a "global conversation". It's "what's happening."<br><br>

<img src="https://github.com/computationaljournalism/columbia2018/raw/master/images/tw.jpg" style="width: 65%; border: #000000 1px outset;">
<br><br>

There are **so** many questions about trends and how they are implemented on the various social media platforms. Today we are going to bring out some of the ways the idea of a trend has been made computational, and then we'll explore a little bit what trending looks like. 

**The usual start-up**

We spent a fair bit of time looking at Twitter data about #schumershutdown versus #trumpshutdown, the idea being that if one becomes a trending topic then it is "what's happening." It is what Twitter (and by extension, some will assume) the country is thinking. It's the Democrats! It's the Republicans! So, let's continue our look at trends.

We will review briefly the Twitter Trending API. We will then step it up a bit and tap into Twitter as a source of information about the topics we're seeing.

We start where we ended last time - by getting our Twitter API initialized:

In [ ]:
# grab your keys from a previous notebook or https://apps.twitter.com

consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

In [ ]:
# before we can make Twitter API calls, we need to initialize a few things...
from tweepy import OAuthHandler, API

# setup the authentication
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# create an object we will use to communicate with the Twitter API
api = API(auth)

**Where on Earth?**

Twitter has a few Trends-related APIs including one that allows you to see which locations have Trending data.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;https://developer.twitter.com/en/docs/trends/locations-with-trending-topics/api-reference

Below, we use a simple service or API to list the different locations for which Twitter computes trends. We have seen a few of these locations in Trendsmap in class, but what's the complete set look like? We'll call the `trends_available` API through the Tweepy API object.

In [ ]:
# get all of the trends locations - store the result in a 
# variable called 'avail_locations' 

avail_locations = api.trends_available()
print(type(avail_locations))

The API via Tweepy has returned a Python object representing the locations. It is a list, which means we can access entries by specifying their number. So, like this for the first one (we start counting at zero)...

In [ ]:
avail_locations[0]

... and this for the second...

In [ ]:
avail_locations[1]

... and we can see how many entries we have using the function `len()`. Remember, lists store data in sequential order, in a, well, list. So there is a first item (index 0) a second item (index 1) and so on up until the last. 

In [ ]:
len(avail_locations)

With 467 locations have a lot to choose from! So, in the space below, print out a couple more entries from the list `avail_locations`. That is, select indices other than 0 or 1 and print out the dictionaries associated with these new locations.

In [ ]:
# print out a couple locations we haven't seen (ie not location with index 0 or 1)



Now, each entry in the `avail_locations` list is again a Python object. What kind? Here we check on the fifth entry in the list (the one having index 4).

In [ ]:
print(type(avail_locations[4]))

It's a dictionary. So `avail_locations` is a list of dictionaries. Remember that dictionaries don't store data sequentially, but by name (essentially). We use the method `keys()` to tell us under what names the data in each dictionary was stored.

In [ ]:
avail_locations[4].keys()

We see keys like `name` and `country`. These are data we can access from each location...

In [ ]:
avail_locations[4]["country"]

Take a second to let that notation sink in. We have `avail_locations` as a list. The selection or subset `[4]` focuses on the fifth entry in the list (remember we start counting at 0). That fifth entry is a dictionary and the selection `["country"]` picks just the data from the dictionary stored under the label `"country"`. 

If this is confusing we can unwrap it in a couple steps. The code below and the code above are essentially the same. Which is easier for you to read?

In [ ]:
location = avail_locations[4]
location["country"]

**Loops**

Now, we can process these data by creating a loop. Using the notion we saw above where we first created a variable, `location`, and then subset out the data we wanted, `country`, we can do something similar in a loop.

Below we loop over all the entries in the list `avail_location`, each pass through the loop assigning `location` to the next entry. We then print out the entry.

In [ ]:
# avail_locations is a list so we can loop through and print them out
for location in avail_locations:
    print(location)

OK finally, using the same line of reasoning we had before, copy the code above to the cell below and print out not the entire `location` dictionary, but instead just a field of your choice.

In [ ]:
# Put your code here - print out just one field




In [ ]:
# If you get ambitious, print out two pieces of information 
# like, say, place name and country name -- you can do this by passing not
# just one object to print() but two, separated by a comma (as we have in other notebooks)




**Data Frames**

As we saw in class, we can think of `avail_locations` as a kind of unrolled table. Each  element in the list `avail_locations` is a location and can be thought of as a **row in a table**. Then, the named components in the dictionary for a location represent the **column entries in a row**. 

There are services online that will do this, just to bring the point home. Copy a few of the dictionaries above and paste them into [this conversion site](https://json-csv.com/) and see the result.

So, with something that looks like a table, Pandas can make short work of it and take each entry in `avail_location` make it a row with columns coming from the keys in the dictionaries. 

Two lines, using the function `DataFrame` and not `read_csv` this time.

In [ ]:
from pandas import DataFrame

locations = DataFrame(avail_locations)
locations

Now, as a data frame there are things you can do easily like search for all the places in a country or all the cities that have the same name.

In [ ]:
# use the data frame `locations` and subset out all the places in Portugal




**The NY State of Mind**

Looking more deeply at the `locations` data frame, of all the data represented, there seems to be an ID associated with each place. As we mentioned these are `woeid`'s which stand for "where on earth" id. You can [read about their use in Twitter here](https://blog.twitter.com/engineering/en_us/a/2010/woeids-in-twitters-trends.html
).

<br><br>
<img src="https://github.com/computationaljournalism/columbia2018/raw/master/images/wo.jpg" style="width: 65%; border: #000000 1px outset;"/>
<br><br>

Now...let's get the current trends for New York. Below we subset the dataframe `locations` to keep just two columns -- the "name" of the place and it's woeid. We then keep only the rows that are associated with the "US" `countryCode`. 

Let's creep up on it. First, keep just the `name` and `woeid` columns.

In [ ]:
locations[["name","woeid"]].head()

Next, subset out just the places with `countryCode` being the United States or "US".

In [ ]:
locations[["name","woeid"]][locations["countryCode"]=="US"]

Make sure you understand this!!! If not <a href=mailto:markh@columbia.edu>email me!</a>

So we now have the New York City woeid, 2459115, and can use it together with the `trends_place` API offered by Twitter, accessed through Tweepy. You can read about the `trends_place` API from Twitter in its developer documentation. [Have a look!](https://developer.twitter.com/en/docs/trends/trends-for-location/api-reference/get-trends-place)

In [ ]:
# get trends for New York

woeid = 2459115
ny_trends = api.trends_place(woeid)

print(type(ny_trends))

You see that this is a list again. Great. We know what to ask. How many objects are in the list?

In [ ]:
len(ny_trends)

As we saw in class, this list has only one object. It looks like the Twitter API used to let you ask for multiple places (and so you could have a list of 10 or 20 or whatever trends results, depending on how many places you asked for), but now you can only request one at a time. So we get a list of one object. 

To see it, we use index 0 (we start counting from 0).

In [ ]:
ny_trends[0]

 Again, the object inside our list is a dictionary. Great. We should probably  now just take the dictionary out of the list (the enclosing list is a little silly anyway since it only holds one object). Below we're just going to rename `ny_trends` to be the single dictionary in the list.

In [ ]:
woeid = 2459115
ny_trends = api.trends_place(woeid)[0]

print(type(ny_trends))

Now, since we used `[0]` to select the single entry in the list that was returned to us by the Twitter API, `ny_trends` is now a dictionary. We can use `.keys()` to see what it contains. 

In [ ]:
ny_trends.keys()

Tell me what the keys are and what kind of thing is stored under each key (what kind of object) and also what data is represented in each object.

In [ ]:
# Put your code here. Find the keys for the ny_trends dictionary and explain the structute 
# of each object (is it a number or a string or a list or a dictionary?) and what data 
# it represents




Put any text here

.

.

.

The `trends` key stores a list and that list is made up of dictionaries. (Yes, I am giving away some of the answer to the question above - bonus!) Hmm, we just saw that with the `avail_locations` object. 

So again, the data under the `trends` key looks like a table in disguise. It is a list with each element representing a row, and the dictionary names refer to the column elements. So, we can use Pandas to read it into a data frame. Easy Peasy.

In [ ]:
from pandas import DataFrame

# turn the list of dictionaries from the trends API into a dataframe
ny = DataFrame(ny_trends["trends"])
ny

**Searching**

If you wanted to sample something of the discussion around a topic that's trending, you can use the `query` column in the ny data frame, say, to post a search to Twitter. We did this last time, using the `search` API, but here it is again. We are looking backwards in time, from "now" back, finding tweets that contain our query, our trending topic. 

In an upcoming lecture we will teach you how to listen to the stream in realtime, pulling tweets as they happen. But for now, let's see what the conversation around "MyFirstVehicle" is.

In [ ]:
# Pass the query to the search() method and collect a list of tweets.
# we have seen tweet objects from Tweepy already. Here they are again!
# we get the date the tweeet was posted with tweet.created_at and the 
# text of the tweet with tweet.text, for example.

tweets = api.search("%23MyFirstVehicleWas")

for tweet in tweets:
    print(tweet.text)

### Trends in Twitter###

For today's drill let's look at our trend data again. Recall that we collected it every 15 minutes over a wide geography and chose our time window so as to cover the State of the Union address. You can [download the data](https://www.dropbox.com/s/jl1jrmtv0w4pefq/twitter_trending_topics_2018_SOTU_v3.csv.gz), uncompressit, and place it in the same folder as this notebook.

In [ ]:
from pandas import read_csv, set_option
set_option('display.max_rows', 100)

In [ ]:
trends = read_csv('twitter_trending_topics_2018_SOTU_v3.csv',low_memory=False)
trends.shape

In [ ]:
trends.head(100)

The data are sorted first by city and then by time, running from oldest to newest topic. The display of the first 100 lines shows this structure clearly. After 50 trends from 4:15 in the morning on January 30 (UTC), we jump to the trends from 30 minutes after four in Algeria.

We can look at the first and the last entries to get a sense of the period of our data collection from Twitter.

In [ ]:
trends.head(1)

In [ ]:
trends.tail(1)

So, data were collected from twitter from 11:15pm NY time on January 29 until just before 6:45pm NY time on January 31st. So we have 32 hours or so of data collection.

The columns include the time the trend was collected, the country we are referring to, the city, the rank of the topic in that time period in that city and country, and then the topic name. It looks like only the US has cities in this data set and the other nations only have national-level trends.

We see that there are 63 unique countries and we can look at how many trends we have from each...

In [ ]:
trends["country"].value_counts()

And here are the US cities...

In [ ]:
trends["city"].value_counts()

**Having a deeper look**

With the trends in a simple data frame, we can now ask questions like, "What was the top-ranked topic in El Paso during the time we collected data?" Let's creep up on it. First, just the topics from El Paso. We take the `topic_name` column from the `trends` data frame and keep only those rows where the corresponding `city` is El Paso.

In [ ]:
trends["topic_name"][trends["city"]=="El Paso"]

Now, let's add a condition so that the rows we keep are not just associated with El Paso, but they are also ranked number 1 in the city. We join two boolean conditions with an `&` -- it means both have to be true. (The `|` "or" symbol is true if either of the two conditions is true.) We also have to put the conditions in parentheses to keep them separate -- so Pandas can clearly see it's this condition AND this other condition. (It's easier to read also.)

In [ ]:
trends["topic_name"][(trends["city"]=="El Paso") & (trends["rank"]==1)]

The natural next thing to do is to count up how many times each topic was ranked number one. We can do this with `value_counts()`.

In [ ]:
trends["topic_name"][(trends["city"]=="El Paso") & (trends["rank"]==1)].value_counts()

So, the State of the Union was at the top the most often. Then I-10. What was that? When did that trend? We can see by simply making a plot. On the x-axis we will put the time the trend was recorded, `datetime` and on the y-axis the different trends that were at number 1.

The `myplot_parts` below defines an x-y plot and the `mylaout` gives you the height and width of the plot and mucks with the margins a little (we are doing this so that the labels on the left of the plot have enough room to show up).

In [ ]:
# the good old plotly plot

from plotly.plotly import iplot, sign_in
import plotly.graph_objs as go 

sign_in("cocteautt","8YLww0QuMPVQ46meAMaq")

trendy_tops = trends[(trends["city"]=="El Paso") & (trends["rank"]==1)]

myplot_parts = [go.Scatter(x=trendy_tops["datetime"],y=trendy_tops["topic_name"],mode="markers")]
mylayout = go.Layout(autosize=False, width=800,height=400,margin=go.Margin(l=150,r=50,b=100,t=100,pad=4))
myfigure = go.Figure(data = myplot_parts, layout = mylayout)
iplot(myfigure,filename="elpaso")

We see clearly that the Grammy's came first. Remember the times here are 5 hours ahead of NYC time and 7 hours ahead of El Paso. So that 6am on Jan 30 is 11pm on Jan 29 in El Paso. The Grammy's happened on the 28th so El Paso was still buzzing  a day later. 

I-10 comes next. Why might that be trending? It's 1pm UTC and that's 7 hours ahead of El Paso, so it's 6am El Paso time. Might be an accident? Traffic reports? Any idea?

Now, considering only the very top trending item might be too limiting. We can ask topics in the top 3 with a small change in our code. Here's the same plot, but we change the `==1` to be `<=3` which will give us ranks that are 3, 2 or 1. We also changed the height of the plot (in the `mylayout` piece) to be 600 pixels. When you have a lot of categories, they sometimes need more room otherwise everyting looks squished and plotly will drop off some of the labels.

In [ ]:
trendy_tops = trends[(trends["city"]=="El Paso") & (trends["rank"]<=3)]

myplot_parts = [go.Scatter(x=trendy_tops["datetime"],y=trendy_tops["topic_name"],mode="markers")]
mylayout = go.Layout(autosize=False, width=800,height=600,margin=go.Margin(l=150,r=50,b=100,t=100,pad=4))
myfigure = go.Figure(data = myplot_parts, layout = mylayout)
iplot(myfigure,filename="elpaso")

Here you notice that we might want to do some regularization to the trends. That is, we might want to make the trends lowercase, at least so that #GRAMMYS and #grammys are grouped on the same line. 

To do that, we can access the `.str` portion of a data frame's columns when those columns contain string data. Have a look at the syntax.

In [ ]:
# regular and noisy

trendy_tops["topic_name"].head(10)

In [ ]:
# accessing the "string" methods we learned on day 1

trendy_tops["topic_name"].str.lower().head(10)

See the difference? We can use this to create a new column in the original `trends` data frame where we can store "clean" trend names. There might be other things we want to do to these strings, and we'd lump all the transformations into the entries of this new column and use it when we need to compare trends in time (since in time caps change, etc). 

We create a new column in a data frame simply by using its name on the left side of an `=` sign. Here we create the column called `clean_topic_name`.

In [ ]:
trends["clean_topic_name"] = trends["topic_name"].str.lower()
trends.head()

And now make the plot from El Paso. Notice now that in the second line we are using `clean_topic_name` and not `topic_name`.

In [ ]:
trendy_tops = trends[(trends["city"]=="El Paso") & (trends["rank"]<=3)]

myplot_parts = [go.Scatter(x=trendy_tops["datetime"],y=trendy_tops["clean_topic_name"],mode="markers")]
mylayout = go.Layout(autosize=False, width=800,height=600,margin=go.Margin(l=150,r=50,b=100,t=100,pad=4))
myfigure = go.Figure(data = myplot_parts, layout = mylayout)
iplot(myfigure,filename="elpaso")

Better! Let's try a different city... 

In [ ]:
trendy_tops = trends[(trends["city"]=="Honolulu") & (trends["rank"]<=3)]

myplot_parts = [go.Scatter(x=trendy_tops["datetime"],y=trendy_tops["clean_topic_name"],mode="markers")]
mylayout = go.Layout(autosize=False, width=800,height=600,margin=go.Margin(l=150,r=50,b=100,t=100,pad=4))
myfigure = go.Figure(data = myplot_parts, layout = mylayout)
iplot(myfigure,filename="honolulu")

See the difference? There are certainly different topics trending over this period. There are some similarities but there are also local eccentricities.

Pick two more cities and look at either the top-ranking trend or among the top 3 or 5 or even 10 (since 10 show up in the web interface) and make plots. Compare the cities we've seen so far. If you notice that there are rows of dots that don't have a label at the left, increase the height of the plot until those labels appear. 

In [ ]:
# Put code here for City 1




In [ ]:
# Pute code here for City 2




We can see some similarities and some differences. One thing I noticed was the prevalence of 'james harden' in the top 10. Who is he and why would he be trending? (If you don't know, dust off Google.) To examine where his name trended, we can make a new data frame with just "james harden" in the top 10 topics in US cities. 

Here we go. Call the new subsetted data frame `jh`. Here we are again using a string method through `.str`, this time `contains()`. It returns `True` if the string entry in the row contains the given pattern and `False` otherwise. You might want to go back and review all the things you can do to strings and, as a result, apply after `.str` in a data frame.

In [ ]:
jh = trends[trends["clean_topic_name"].str.contains("james harden") & 
                 (trends["country"]=="United States") &
                 (trends["rank"]<=10)]
jh.head()

Or maybe have a plot instead. Here we use a very tall plot because there are a lot of cities to display. What do you see? Why was James Harden trending and where was he in the top 10 the longest?

In [ ]:
jh = trends[trends["clean_topic_name"].str.contains("james harden") & 
            (trends["country"]=="United States") &
            (trends["rank"]<=10)]

myplot_parts = [go.Scatter(x=jh["datetime"],y=jh["city"],mode="markers")]
mylayout = go.Layout(autosize=False, width=800,height=1300,margin=go.Margin(l=150,r=50,b=100,t=100,pad=4))
myfigure = go.Figure(data = myplot_parts, layout = mylayout)
iplot(myfigure,filename="harden")

Now, instead of going straight to a plot, you could also spot something (like I-10) and see what other cities it was in the top 10 for. Here we create a new data frame called `tester` that we can use to see which cities had a particular topic in the top 10 and for how many of our 15 minute periods.

In [ ]:
tester = trends[trends["clean_topic_name"].str.contains("i-10") & 
               (trends["country"]=="United States") &
               (trends["rank"]<=10)]

tester["city"].value_counts()

Here El Paso topped the list with 132 periods in the top 10 (132\*15/60 = 33 hours). Baton Rouge was close behind. "alex smith" had a pattern similar to Harden...

In [ ]:
tester = trends[trends["clean_topic_name"].str.contains("alex smith") & 
               (trends["country"]=="United States") &
               (trends["rank"]<=10)]

tester["city"].value_counts().head(25)

Now, go back and forth, looking at places (not necessarily US cities) and see which topics appear and when. Then look across places and see where they spread. 

In [ ]:
# Your code here




In [ ]:
# and here... etc




**And Finally...**

Write something down about what you learned. What was going on during the SOTU? Or maybe something else you observed. Don't stick to the parameters we gave you. Maybe look deeper into the top 50... Maybe track a single topic over time in a fixed city and plot its rank. Write something down about what you saw and include the code to back it up.

In [ ]:
#your code




Your text

.

.

.